In [69]:
import pandas as pd
import numpy as np
import os
import operator
import random

login = os.getlogin()
data_dir = f'/mnt/container-nle-tcr/tc-hard-data/'

In [70]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
set_random_seed(42)

In [71]:
def print_stats(df, mhc="allele"):

    if mhc == "allele":
        mhc_col = "mhc.a"
    elif mhc == "seq":
        mhc_col = "mhc.seq"
    else:
        raise NotImplementedError

    print('Total samples:', len(df))

    temp = df.copy()
    temp = temp[["cdr3.beta", "antigen.epitope", 'label']].dropna().drop_duplicates()
    pos = temp[temp['label']==1]
    neg = temp[temp['label']==0]
    print('\n With CDR3b + pep: ',len(temp))
    print("Non-binding samples: ", len(neg))
    print("Binding samples: ", len(pos))

    temp = df.copy()
    temp = temp[["cdr3.beta", "antigen.epitope", 'label', mhc_col]].dropna().drop_duplicates()
    pos = temp[temp['label']==1]
    neg = temp[temp['label']==0]
    print(f'\n With CDR3b + pep + MHC {mhc}: ',len(temp))
    print("Non-binding samples: ", len(neg))
    print("Binding samples: ", len(pos))

    temp = df.copy()
    temp = temp[["cdr3.beta", "antigen.epitope", 'label', "cdr3.alpha"]].dropna().drop_duplicates()
    pos = temp[temp['label']==1]
    neg = temp[temp['label']==0]
    print(f'\n With CDR3b + pep + CDR3a {mhc}: ',len(temp))
    print("Non-binding samples: ", len(neg))
    print("Binding samples: ", len(pos))

    temp = df.copy()
    temp = temp[["cdr3.alpha", "cdr3.beta", "antigen.epitope", 'label', mhc_col]].dropna().drop_duplicates()
    pos = temp[temp['label']==1]
    neg = temp[temp['label']==0]
    print(f'\n With CDR3b + pep + CDR3a + MHC {mhc}: ',len(temp))
    print("Non-binding samples: ", len(neg))
    print("Binding samples: ", len(pos))

# VDJdb

In [72]:
vdjdb_df = pd.read_csv(data_dir+'vdjdb-2021-09-05/vdjdb_full.txt', sep="\t")
vdjdb_df = vdjdb_df[vdjdb_df['species'] == 'HomoSapiens']

/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (20,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
# filter VDJdb based on confidence score
vdjdb_df = vdjdb_df[vdjdb_df['vdjdb.score'] >= 1]

In [74]:
# only select class I MHCs
vdjdb_df = vdjdb_df[vdjdb_df['mhc.class'] == 'MHCI']

In [75]:
vdjdb_df = vdjdb_df[["cdr3.alpha", "cdr3.beta", "mhc.a", "antigen.epitope", "v.alpha", "j.alpha", "v.beta", "d.beta", "j.beta"]]

# remove rows with NaN CDR3 beta and peptides (for CDR3 alpha and MHC we accept NaN)
vdjdb_df.dropna(subset=["cdr3.beta", "antigen.epitope"], inplace=True)

In [76]:
# filter for non-aa characters in CDR3 beta sequences;
# do the same in CDR3 alpha if they're not NeN (keep the NaN alpha)
alpha_nan_or_is_amino = operator.or_(vdjdb_df['cdr3.alpha'].str.match('^[A-Z]+$') == True, vdjdb_df['cdr3.alpha'].isna())
beta_is_amino = vdjdb_df['cdr3.beta'].str.match('^[A-Z]+$') == True
vdjdb_df = vdjdb_df.loc[alpha_nan_or_is_amino & beta_is_amino]

In [77]:
# if CDR3 alpha and CDR3 beta do not start with `C`, add it
vdjdb_df['cdr3.beta'] = vdjdb_df['cdr3.beta'].apply(lambda x: 'C' + x if not pd.isnull(x) and x[0] != 'C' else x)
vdjdb_df['cdr3.alpha'] = vdjdb_df['cdr3.alpha'].apply(lambda x: 'C' + x if not pd.isnull(x) and x[0] != 'C' else x)

# filter for unclear/PTM epitopes
vdjdb_df = vdjdb_df.loc[(vdjdb_df['antigen.epitope'].str.match('^[A-Z]+$') == True)].reset_index(drop=True)

In [78]:
# filter for duplicates
vdjdb_df = vdjdb_df.drop_duplicates(subset=['cdr3.alpha', 'cdr3.beta', 'antigen.epitope', 'mhc.a'], keep='first')

In [79]:
# add label - all samples are positive in this dataset
vdjdb_df['label'] = 1

# mark negative source
vdjdb_df["negative.source"] = np.nan

# specify license
vdjdb_df['license'] = "vdjdb_license"

In [80]:
print_stats(vdjdb_df)

Total samples: 3706

 With CDR3b + pep:  3453
Non-binding samples:  0
Binding samples:  3453

 With CDR3b + pep + MHC allele:  3554
Non-binding samples:  0
Binding samples:  3554

 With CDR3b + pep + CDR3a allele:  882
Non-binding samples:  0
Binding samples:  882

 With CDR3b + pep + CDR3a + MHC allele:  920
Non-binding samples:  0
Binding samples:  920


# Combine the datasets

In [81]:
df = pd.concat([vdjdb_df]).reset_index(drop=True)
len(df)

3706

In [82]:
df = df.drop_duplicates(subset=["cdr3.alpha", "cdr3.beta", "mhc.a", "antigen.epitope", "label"], keep="first").reset_index(drop=True)
len(df)

3706

In [83]:
# Remark: we observe that certain samples are labelled in an inconsistent way
# This is an argument for not considering negative samples from IEDB. but only randomized negative samples
q = df[df.duplicated(subset=["cdr3.alpha", "cdr3.beta", "mhc.a", "antigen.epitope"], keep=False)]
print(len(q))
# Example:
q[q["antigen.epitope"] == "EVLPFFLFF"]

0


,cdr3.alpha,cdr3.beta,mhc.a,antigen.epitope,v.alpha,j.alpha,v.beta,d.beta,j.beta,label,negative.source,license


# Fix the HLAs

In [84]:
# remove the "mutant"
mutant_dict = {}
for mhc in df['mhc.a'].unique():
    if type(mhc) == str and "mutant" in mhc:
        mutant_dict[mhc] = mhc.split(" ")[0].replace(",", "")
df['mhc.a'] = df['mhc.a'].apply(lambda x: mutant_dict[x] if x in mutant_dict.keys() else x)

In [85]:
# trim HLA resolution when > 4
import re

match = "\d\d:\d\d:\d\d"
higher_resolution_dict = {}
for mhc in df['mhc.a'].unique():
    if type(mhc) == str:
        r = re.search(match, mhc)
        if r:
            s = mhc.split(":")
            higher_resolution_dict[mhc] = s[0]+":"+s[1]

df['mhc.a'] = df['mhc.a'].apply(lambda x: higher_resolution_dict[x] if x in higher_resolution_dict.keys() else x)

In [86]:
# duplicate rows when ther are multiple HLAs
multiple_hlas_dict = {}
for mhc in df['mhc.a'].unique():
    if type(mhc) == str:
        if "," in mhc:
            multiple_hlas_dict[mhc] = mhc.replace(" ", "").split(",")

rows_to_drop = []
peps = []
cdr3bs = []
cdr3as = []
mhcs = []
labels = []
neg_source = []
v_a =[]
v_b =[]
j_a =[]
j_b =[]
d_b =[]

for index, row in df.iterrows():
    if row['mhc.a'] in multiple_hlas_dict.keys():
        rows_to_drop.append(index)
        for mhc in multiple_hlas_dict[row['mhc.a']]:
            mhcs.append(mhc)
            peps.append(row["antigen.epitope"])
            cdr3bs.append(row["cdr3.beta"])
            cdr3as.append(row["cdr3.alpha"])
            labels.append(row["label"])
            neg_source.append(row["negative.source"])
            v_a.append(row["v.alpha"])
            v_b.append(row["v.beta"])
            j_a.append(row["j.alpha"])
            j_b.append(row["j.beta"])
            d_b.append(row["d.beta"])

df = df.drop(rows_to_drop)

df_to_add = pd.DataFrame({
    "antigen.epitope": peps,
    "mhc.a": mhcs,
    "cdr3.beta": cdr3bs,
    "cdr3.alpha": cdr3as,
    "label": labels,
    "negative.source": neg_source,
    "v.alpha": v_a,
    "v.beta": v_b,
    "j.alpha": j_a,
    "j.beta": j_b,
    "d.beta": d_b
})

df = pd.concat([df, df_to_add]).reset_index(drop=True)

In [87]:
# at this point, try to normalize the MHC allele
from mhcnames.normalization import normalize_allele_name

def normalize(x):
    try:
        return normalize_allele_name(x)
    except:
        return np.nan

df["mhc.a"] = df["mhc.a"].apply(lambda x: normalize(x))

# Add the reference ("pseudo") sequences to the dataframe

In [88]:
class1_df = pd.read_csv(data_dir+'mhc-sequences/MHC_pseudo.dat', sep="\s+|\t+|\s+\t+|\t+\s+")
class1_df["mhc"] = class1_df["mhc"].apply(lambda x: normalize(x))
class1_df = class1_df.dropna()
class1_dict = dict(zip(class1_df.mhc, class1_df.sequence))

class2_df = pd.read_csv(data_dir+'mhc-sequences/pseudosequence.2016.all.X.dat', sep="\t")
class2_df["mhc"] = class2_df["mhc"].apply(lambda x: normalize(x))
class2_df = class2_df.dropna()
class2_dict = dict(zip(class2_df.mhc, class2_df.sequence))

mhc_2_seq = {**class1_dict, **class2_dict}
df["mhc.seq"] = df["mhc.a"].apply(lambda x: mhc_2_seq[x] if x in mhc_2_seq.keys() else np.nan)

df = df.reset_index(drop=True).drop_duplicates(subset=["cdr3.alpha", "cdr3.beta", "mhc.seq", "mhc.a", "antigen.epitope", "label"])

/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [89]:
print_stats(df, mhc="seq")

print("\n NaN MHC allele info: ", sum(df["mhc.a"].isna()))
print("Good MHC allele info: ", sum(~df["mhc.a"].isna()))

print("\n NaN MHC sequences: ", sum(df["mhc.seq"].isna()))
print("Good MHC sequences: ", sum(~df["mhc.seq"].isna()))

print("\n NaN CDR3 alpha sequences: ", sum(df["cdr3.alpha"].isna()))
print("Good CDR3 alpha sequences: ", sum(~df["cdr3.alpha"].isna()))

Total samples: 3635

 With CDR3b + pep:  3453
Non-binding samples:  0
Binding samples:  3453

 With CDR3b + pep + MHC seq:  3388
Non-binding samples:  0
Binding samples:  3388

 With CDR3b + pep + CDR3a seq:  882
Non-binding samples:  0
Binding samples:  882

 With CDR3b + pep + CDR3a + MHC seq:  853
Non-binding samples:  0
Binding samples:  853

 NaN MHC allele info:  0
Good MHC allele info:  3635

 NaN MHC sequences:  88
Good MHC sequences:  3547

 NaN CDR3 alpha sequences:  2749
Good CDR3 alpha sequences:  886


# Add negative samples via randomization

In [90]:
def sample_negatives(source_df):
    source_p_mhc = source_df[["antigen.epitope", "mhc.a", "mhc.seq"]]
    source_cdr3b = source_df["cdr3.beta"].dropna()
    source_cdr3a = source_df["cdr3.alpha"].dropna()
    
    beta_gene_df = source_df[["cdr3.beta", "j.beta", "v.beta", "d.beta"]].drop_duplicates()
    cdr3b_2_jb = dict(zip(beta_gene_df["cdr3.beta"], beta_gene_df["j.beta"]))
    cdr3b_2_vb = dict(zip(beta_gene_df["cdr3.beta"], beta_gene_df["v.beta"]))
    cdr3b_2_db = dict(zip(beta_gene_df["cdr3.beta"], beta_gene_df["d.beta"]))

    alpha_gene_df = source_df[["cdr3.alpha", "j.alpha", "v.alpha"]].drop_duplicates()
    cdr3a_2_ja = dict(zip(source_df["cdr3.alpha"], source_df["j.alpha"]))
    cdr3a_2_va = dict(zip(source_df["cdr3.alpha"], source_df["v.alpha"]))

    # sample negative samples, so that we have 2x negatives w.r.t. positives
    N = 2
    
    temp_df = pd.concat([
        source_p_mhc.sample(n=len(source_df), replace=False)
        for i in range(N)
    ])
    
    if len(source_cdr3a) > 0:
        temp_df["cdr3.alpha"] = np.concatenate([
            np.random.choice(source_cdr3a, len(source_df), replace=False)
            for i in range(N)
        ], axis=0)
    else:
        temp_df["cdr3.alpha"] = np.nan

    temp_df["cdr3.beta"] = np.concatenate([
        np.random.choice(source_cdr3b, len(source_df), replace=False)
        for i in range(N)
    ], axis=0)

    # this step ensures that the randomization did not create random samples,
    # which are equal to the positive ones
#     len_pre = len(temp_df)
#     temp = source_df[["antigen.epitope", "mhc.a", "mhc.seq", "cdr3.alpha", "cdr3.beta"]]
#     temp = source_df[["antigen.epitope", "mhc.a", "mhc.seq", "cdr3.beta"]]
#     temp = source_df[["antigen.epitope", "cdr3.beta"]]
#     temp_df = pd.merge(temp_df, temp, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
#     print(f"Dropping {len_pre - len(temp_df)} samples, which have positive (pep,CDR3b) pairs")

    # assign negative label
    temp_df["label"] = 0

    # mark negative samples
    temp_df["negative.source"] = "randomized"
    
    # assign V, D, J genes
    temp_df["v.alpha"] = temp_df["cdr3.alpha"].apply(lambda x: cdr3a_2_va.get(x, np.nan))
    temp_df["j.alpha"] = temp_df["cdr3.alpha"].apply(lambda x: cdr3a_2_ja.get(x, np.nan))
    temp_df["v.beta"] = temp_df["cdr3.beta"].apply(lambda x: cdr3b_2_vb[x])
    temp_df["j.beta"] = temp_df["cdr3.beta"].apply(lambda x: cdr3b_2_jb[x])
    temp_df["d.beta"] = temp_df["cdr3.beta"].apply(lambda x: cdr3b_2_db[x])

    return temp_df

In [91]:
samp_neg_df = []

# we only sample negatives starting from positive samples
pos_df = df[df["label"] == 1]

# case 1: cdr3 alpha and mhc are available
source_df = pos_df.dropna(subset=["cdr3.alpha", "mhc.seq"])
temp_df = sample_negatives(source_df)
samp_neg_df.append(temp_df)

# case 2: cdr3 alpha available, mhc unknown
source_df = pos_df.dropna(subset=["cdr3.alpha"])
source_df = source_df[source_df["mhc.seq"].isna()]
temp_df = sample_negatives(source_df)
samp_neg_df.append(temp_df)

# case 3: cdr3 alpha unknown, mhc is available
source_df = pos_df.dropna(subset=["mhc.seq"])
source_df = source_df[source_df["cdr3.alpha"].isna()]
temp_df = sample_negatives(source_df)
samp_neg_df.append(temp_df)

# case 3: cdr3 alpha unknown, mhc unknown
source_df = pos_df[pos_df["mhc.seq"].isna()]
source_df = source_df[source_df["cdr3.alpha"].isna()]
temp_df = sample_negatives(source_df)
samp_neg_df.append(temp_df)

samp_neg_df = pd.concat(samp_neg_df)
samp_neg_df = samp_neg_df.drop_duplicates(
    subset=['cdr3.alpha', 'cdr3.beta', 'mhc.a', 'antigen.epitope', 'label',
       'negative.source', 'mhc.seq']
).reset_index(drop=True)  # we exclude genes in checking for duplicates

# this step ensures that the randomization did not create random samples,
# which are equal to the positive ones
len_pre = len(samp_neg_df)
temp = df[["antigen.epitope", "cdr3.beta"]]
samp_neg_df = pd.merge(samp_neg_df, temp, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
print(f"Dropping {len_pre - len(samp_neg_df)} randomized negative samples, which present binding (pep,CDR3b) pairs.\n")

print_stats(samp_neg_df, "seq")

Dropping 332 randomized negative samples, which present binding (pep,CDR3b) pairs.

Total samples: 6821

 With CDR3b + pep:  6681
Non-binding samples:  6681
Binding samples:  0

 With CDR3b + pep + MHC seq:  6577
Non-binding samples:  6577
Binding samples:  0

 With CDR3b + pep + CDR3a seq:  1644
Non-binding samples:  1644
Binding samples:  0

 With CDR3b + pep + CDR3a + MHC seq:  1597
Non-binding samples:  1597
Binding samples:  0


In [92]:
df = pd.concat([df, samp_neg_df]).drop_duplicates().reset_index(drop=True)

# Save

In [59]:
df.to_csv(data_dir+"tc-hard/ds.vdjdb-high-scores.csv", index=False)

# Stats - considering full dataset

In [60]:
def check_duplicates(df, mhc="a"):
    print("Samples: ", len(df))

    q = df.drop_duplicates(subset=['cdr3.alpha', 'cdr3.beta', 'antigen.epitope', f'mhc.{mhc}'])
    print("Dropping duplicated p+b+a+mhc: ", len(q))
    p = df.drop_duplicates(subset=['cdr3.alpha', 'cdr3.beta', 'antigen.epitope', f'mhc.{mhc}', 'label'])
    print("Dropping duplicated p+b+a+mhc+label: ", len(p))
    
    q = df.drop_duplicates(subset=['cdr3.beta', 'antigen.epitope', f'mhc.{mhc}'])
    print("Dropping duplicated p+b+mhc: ", len(q))
    p = df.drop_duplicates(subset=['cdr3.beta', 'antigen.epitope', f'mhc.{mhc}', 'label'])
    print("Dropping duplicated p+b+mhc+label: ", len(p))
    
    
    q = df.drop_duplicates(subset=['cdr3.alpha', 'cdr3.beta', 'antigen.epitope'])
    print("Dropping duplicated p+b+a: ", len(q))
    p = df.drop_duplicates(subset=['cdr3.alpha', 'cdr3.beta', 'antigen.epitope', 'label'])
    print("Dropping duplicated p+b+a+label: ", len(p))

    q = df.drop_duplicates(subset=['cdr3.beta', 'antigen.epitope'])
    print("Dropping duplicated p+b: ", len(q))
    p = df.drop_duplicates(subset=['cdr3.beta', 'antigen.epitope', 'label'])
    print("Dropping duplicated p+b+label: ", len(p))

In [61]:
print_stats(df, mhc="seq")

print("\n NaN MHC allele info: ", sum(df["mhc.a"].isna()))
print("Good MHC allele info: ", sum(~df["mhc.a"].isna()))

print("\n NaN MHC sequences: ", sum(df["mhc.seq"].isna()))
print("Good MHC sequences: ", sum(~df["mhc.seq"].isna()))

print("\n NaN CDR3 alpha sequences: ", sum(df["cdr3.alpha"].isna()))
print("Good CDR3 alpha sequences: ", sum(~df["cdr3.alpha"].isna()))

Total samples: 10456

 With CDR3b + pep:  10134
Non-binding samples:  6681
Binding samples:  3453

 With CDR3b + pep + MHC seq:  9965
Non-binding samples:  6577
Binding samples:  3388

 With CDR3b + pep + CDR3a seq:  2526
Non-binding samples:  1644
Binding samples:  882

 With CDR3b + pep + CDR3a + MHC seq:  2450
Non-binding samples:  1597
Binding samples:  853

 NaN MHC allele info:  0
Good MHC allele info:  10456

 NaN MHC sequences:  202
Good MHC sequences:  10254

 NaN CDR3 alpha sequences:  7926
Good CDR3 alpha sequences:  2530


In [62]:
# here we accept inconsistent labelling
# as shown above there are some assays which present the same sequences, but with different labels
check_duplicates(df, mhc="seq")

Samples:  10456
Dropping duplicated p+b+a+mhc:  10454
Dropping duplicated p+b+a+mhc+label:  10454
Dropping duplicated p+b+mhc:  10153
Dropping duplicated p+b+mhc+label:  10153
Dropping duplicated p+b+a:  10437
Dropping duplicated p+b+a+label:  10437
Dropping duplicated p+b:  10134
Dropping duplicated p+b+label:  10134


In [67]:
# check no (pep, CDR3b) pairs with both neg and pos labels
q = df[df.duplicated(subset=["antigen.epitope", "cdr3.beta", ], keep=False)]
p = q[q.label==1]
n = q[q.label==0]
p_b_inconsistent = pd.merge(p, n, on=["antigen.epitope", "cdr3.beta"], how='inner')
assert len(p_b_inconsistent) == 0

In [68]:
# considering positive samples + randomized negative samples (i.e. excluding real 
# negative samples), we want check that all CDR3b sequences which appear
# in the negative samples are also appearing in the positive ones
def check_no_beta_only_in_neg(df):
    # this check only concerns the positive + randomized negative samples
    # we exclude the real negatives
    t = df[df["negative.source"] != "mira"]
    t = t[t["negative.source"] != "iedb"]
    t = t[t["negative.source"] != "nettcr-2.0"]

    b_n = set(df_pep_b[df_pep_b.label==0]["cdr3.beta"].unique())
    b_p = set(df_pep_b[df_pep_b.label==1]["cdr3.beta"].unique())
    assert len(b_n - b_p) == 0

df_pep_b = df[["antigen.epitope", "cdr3.beta", "label", "negative.source"]].copy()
check_no_beta_only_in_neg(df_pep_b)

df_pep_b_mhc = df[["antigen.epitope", "cdr3.beta", "mhc.seq", "label", "negative.source"]].dropna().copy()
check_no_beta_only_in_neg(df_pep_b_mhc)

df_pep_b_a = df[["antigen.epitope", "cdr3.beta", "cdr3.alpha", "label", "negative.source"]].dropna().copy()
check_no_beta_only_in_neg(df_pep_b_a)

df_pep_b_a_mhc = df[["antigen.epitope", "cdr3.beta", "cdr3.alpha", "mhc.seq", "label", "negative.source"]].dropna().copy()
check_no_beta_only_in_neg(df_pep_b_a_mhc)